In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import networkx as nx
import pygraphviz
from networkx.drawing.nx_agraph import graphviz_layout
from networkx.drawing.nx_agraph import to_agraph 
import matplotlib.pyplot as plt
from pylab import hist
import random
from collections import Counter
import operator
import itertools
import cPickle as pickle
import warnings
warnings.filterwarnings("ignore")

import gzip
import matplotlib.dates as mdates

In [2]:
#remove scientific notation
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format

In [5]:
#pull as CSV files together in one df
df = pd.concat(map(pd.read_csv, ['rec_data1.csv', 'rec_data2.csv','rec_data3.csv','rec_data4.csv','rec_data5.csv']))

In [6]:
#pull a CSV files of onehot encoding of categories
des = pd.concat(map(pd.read_csv, ['CatDes.csv']))

In [7]:
des.info()
des.describe

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 3 columns):
cat_code          30 non-null int64
category          30 non-null object
super_category    30 non-null object
dtypes: int64(1), object(2)
memory usage: 792.0+ bytes


<bound method DataFrame.describe of     cat_code              category    super_category
0          1              Artprint          Wall Art
1          2                  Baby           Apparel
2          3                Bangle           Jewelry
3          4               Bedroom              Home
4          5                Canvas          Wall Art
5          6               Dog Tag           Jewelry
6          7              Frameart          Wall Art
7          8     Galaxy Flexi Case  Tech Accessories
8          9      Galaxy Slim Case  Tech Accessories
9         10     Galaxy Tough Case  Tech Accessories
10        11                  Hats       Accessories
11        12     IPhone Flexi Case  Tech Accessories
12        13      IPhone Slim Case  Tech Accessories
13        14     IPhone Tough Case  Tech Accessories
14        15              Necklace           Jewelry
15        16  Phone Case - Samsung  Tech Accessories
16        17           Pillow_Case              Home
17        

In [8]:
#obtain info on dataset 
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1185994 entries, 0 to 225993
Data columns (total 28 columns):
order_date           1185994 non-null object
seller_id            1185994 non-null int64
store_name           1185994 non-null object
customer_key         1185994 non-null object
order_id             1185994 non-null object
locked_units         1185994 non-null object
campaign_tag         1185994 non-null object
campaing_name        1185994 non-null object
front_art_key        842644 non-null object
back_art_key         386963 non-null object
super_category       1185992 non-null object
category             1185992 non-null object
sku                  1185994 non-null object
product_id           1185994 non-null object
color                1185994 non-null object
size                 1185994 non-null object
qty                  1185994 non-null int64
item_index           1185994 non-null int64
items_in_order       1185994 non-null int64
line_item_revenue    1185994 non-null o

,seller_id,qty,item_index,items_in_order
count,1185994.00,1185994.00,1185994.00,1185994.00
mean,5110.07,1.05,1.36,1.73
std,5024.42,0.29,0.83,1.26
min,22.00,0.00,1.00,1.00
25%,22.00,1.00,1.00,1.00
50%,5708.00,1.00,1.00,1.00
75%,11455.00,1.00,1.00,2.00
max,11751.00,33.00,24.00,24.00


In [9]:
#change object to float
df['line_item_revenue'] = (df['line_item_revenue'].str.replace(r'[^-+\d.]', '').astype(float))

In [10]:
#change object to float
df['partner_cost'] = (df['partner_cost'].str.replace(r'[^-+\d.]', '').astype(float))

In [11]:
#change object to float
df['vendor_cost'] = (df['vendor_cost'].str.replace(r'[^-+\d.]', '').astype(float))

In [12]:
#ensure float changes took effect
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1185994 entries, 0 to 225993
Data columns (total 28 columns):
order_date           1185994 non-null object
seller_id            1185994 non-null int64
store_name           1185994 non-null object
customer_key         1185994 non-null object
order_id             1185994 non-null object
locked_units         1185994 non-null object
campaign_tag         1185994 non-null object
campaing_name        1185994 non-null object
front_art_key        842644 non-null object
back_art_key         386963 non-null object
super_category       1185992 non-null object
category             1185992 non-null object
sku                  1185994 non-null object
product_id           1185994 non-null object
color                1185994 non-null object
size                 1185994 non-null object
qty                  1185994 non-null int64
item_index           1185994 non-null int64
items_in_order       1185994 non-null int64
line_item_revenue    1185994 non-null f

,seller_id,qty,item_index,items_in_order,line_item_revenue,partner_cost,vendor_cost
count,1185994.00,1185994.00,1185994.00,1185994.00,1185994.00,1185984.00,1173604.00
mean,5110.07,1.05,1.36,1.73,32.69,10.37,5.08
std,5024.42,0.29,0.83,1.26,14.46,6.69,4.55
min,22.00,0.00,1.00,1.00,0.00,2.35,1.37
25%,22.00,1.00,1.00,1.00,24.99,6.15,1.84
50%,5708.00,1.00,1.00,1.00,27.95,8.35,3.74
75%,11455.00,1.00,1.00,2.00,35.95,13.25,6.37
max,11751.00,33.00,24.00,24.00,1582.35,569.25,62.11


In [13]:
#Create new column for seller margin
df['Seller_Margin'] = df['line_item_revenue'] - df['partner_cost']

In [14]:
#Create new column for GearLaunch margin
df['GL_Margin'] = df['partner_cost'] - (df['qty'] * df['vendor_cost']) 

In [15]:
#ensure new columns are added
df.tail()

,order_date,seller_id,store_name,customer_key,order_id,locked_units,campaign_tag,campaing_name,front_art_key,back_art_key,...,partner_cost,vendor_cost,vendor,ship_state,ship_city,ship_zip,ship_country,region,Seller_Margin,GL_Margin
225989,2019-08-28,5957,WTG Tee,ahBzfmdlYXJsYXVuY2gtaHVichILEghDdXN0b21lchjChd...,OR_AF0P4TP,Y,[],Limited Edition T280819-02,ahBzfmdlYXJsYXVuY2gtaHVicg0LEgNBcnQYgqjRgioM,NaN,...,6.15,1.77,DREAM_JUNCTION,NY,Staten Island,10309-4283,US,Americas,19.84,4.38
225990,2019-09-07,5957,WTG Tee,ahBzfmdlYXJsYXVuY2gtaHVichILEghDdXN0b21lchjz4N...,OR_AK1Y5TC,N,[],Selling Out Fast - Limited Edition,ahBzfmdlYXJsYXVuY2gtaHVicg0LEgNBcnQY09GVnCAM,NaN,...,4.85,3.50,DISTRICT_PHOTO,FL,Deland,32724-4481,US,Americas,19.14,1.35
225991,2019-10-21,5957,WTG Tee,ahBzfmdlYXJsYXVuY2gtaHVichILEghDdXN0b21lchiH8O...,OR_B106WDM,N,[],Selling Out Fast - Limited Edition,ahBzfmdlYXJsYXVuY2gtaHVicg0LEgNBcnQY09GVnCAM,NaN,...,13.15,9.65,CIRCLE_GRAPHICS,GA,Macon,31210-7525,US,Americas,22.84,3.50
225992,2019-12-01,5957,WTG Tee,ahBzfmdlYXJsYXVuY2gtaHVichILEghDdXN0b21lchiQuv...,OR_B8QF9BS,N,[],Selling Out Fast - Limited Edition,ahBzfmdlYXJsYXVuY2gtaHVicg0LEgNBcnQY09GVnCAM,NaN,...,34.55,25.99,CIRCLE_GRAPHICS,WV,Scott Depot,25560-9675,US,Americas,25.44,8.56
225993,2020-01-08,5957,WTG Tee,ahBzfmdlYXJsYXVuY2gtaHVichILEghDdXN0b21lchi95p...,OR_BCYFB44,N,"['featured', 'funny', 'main']",Limited Edition - Selling Out Fast,ahBzfmdlYXJsYXVuY2gtaHVicg0LEgNBcnQYgri-8xkM,NaN,...,6.15,1.81,SPOD,NY,Armonk,10504-1439,US,Americas,18.80,4.34


In [16]:
#remove records with margin <0

df = df[df.Seller_Margin > 0]

In [17]:
#remove records with margin <0

df = df[df.GL_Margin > 0]

In [18]:
#ensure new columns make sense
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1172420 entries, 0 to 225993
Data columns (total 30 columns):
order_date           1172420 non-null object
seller_id            1172420 non-null int64
store_name           1172420 non-null object
customer_key         1172420 non-null object
order_id             1172420 non-null object
locked_units         1172420 non-null object
campaign_tag         1172420 non-null object
campaing_name        1172420 non-null object
front_art_key        829366 non-null object
back_art_key         385657 non-null object
super_category       1172418 non-null object
category             1172418 non-null object
sku                  1172420 non-null object
product_id           1172420 non-null object
color                1172420 non-null object
size                 1172420 non-null object
qty                  1172420 non-null int64
item_index           1172420 non-null int64
items_in_order       1172420 non-null int64
line_item_revenue    1172420 non-null f

,seller_id,qty,item_index,items_in_order,line_item_revenue,partner_cost,vendor_cost,Seller_Margin,GL_Margin
count,1172420.00,1172420.00,1172420.00,1172420.00,1172420.00,1172420.00,1172420.00,1172420.00,1172420.00
mean,5164.14,1.05,1.37,1.73,32.75,10.38,5.08,22.37,5.08
std,5023.52,0.29,0.84,1.26,14.51,6.71,4.55,8.98,2.64
min,22.00,1.00,1.00,1.00,5.45,2.35,1.37,0.30,0.02
25%,22.00,1.00,1.00,1.00,24.99,6.15,1.84,18.80,4.18
50%,5708.00,1.00,1.00,1.00,27.95,8.35,3.74,20.10,4.42
75%,11455.00,1.00,1.00,2.00,35.99,13.25,6.36,23.80,5.87
max,11751.00,33.00,24.00,24.00,1582.35,569.25,62.11,1013.10,161.76


## LIGHTFM

In [19]:
from lightfm import LightFM

In [20]:
#sample code from: https://github.com/aayushmnit/cookbook/blob/master/notebooks/recsys/Recommender.ipynb

import pandas as pd
import numpy as np
from scipy import sparse
from lightfm import LightFM
from sklearn.metrics.pairwise import cosine_similarity

def create_interaction_matrix(df,user_col, item_col, rating_col, norm= False, threshold = None):
    '''
    Function to create an interaction matrix dataframe from transactional type interactions
    Required Input -
        - df = Pandas DataFrame containing user-item interactions
        - user_col = column name containing user's identifier
        - item_col = column name containing item's identifier
        - rating col = column name containing user feedback on interaction with a given item
        - norm (optional) = True if a normalization of ratings is needed
        - threshold (required if norm = True) = value above which the rating is favorable
    Expected output - 
        - Pandas dataframe with user-item interactions ready to be fed in a recommendation algorithm
    '''
    interactions = df.groupby([user_col, item_col])[rating_col] \
            .sum().unstack().reset_index(). \
            fillna(0).set_index(user_col)
    if norm:
        interactions = interactions.applymap(lambda x: 1 if x > threshold else 0)
    return interactions

def create_user_dict(interactions):
    '''
    Function to create a user dictionary based on their index and number in interaction dataset
    Required Input - 
        interactions - dataset create by create_interaction_matrix
    Expected Output -
        user_dict - Dictionary type output containing interaction_index as key and user_id as value
    '''
    user_id = list(interactions.index)
    user_dict = {}
    counter = 0 
    for i in user_id:
        user_dict[i] = counter
        counter += 1
    return user_dict
    
def create_item_dict(df,id_col,name_col):
    '''
    Function to create an item dictionary based on their item_id and item name
    Required Input - 
        - df = Pandas dataframe with Item information
        - id_col = Column name containing unique identifier for an item
        - name_col = Column name containing name of the item
    Expected Output -
        item_dict = Dictionary type output containing item_id as key and item_name as value
    '''
    item_dict ={}
    for i in range(df.shape[0]):
        item_dict[(df.loc[i,id_col])] = df.loc[i,name_col]
    return item_dict

def runMF(interactions, n_components=30, loss='warp', k=15, epoch=30,n_jobs = 4):
    '''
    Function to run matrix-factorization algorithm
    Required Input -
        - interactions = dataset create by create_interaction_matrix
        - n_components = number of embeddings you want to create to define Item and user
        - loss = loss function other options are logistic, brp
        - epoch = number of epochs to run 
        - n_jobs = number of cores used for execution 
    Expected Output  -
        Model - Trained model
    '''
    x = sparse.csr_matrix(interactions.values)
    model = LightFM(no_components= n_components, loss=loss,k=k)
    model.fit(x,epochs=epoch,num_threads = n_jobs)
    return model

def sample_recommendation_user(model, interactions, user_id, user_dict, 
                               item_dict,threshold = 0,nrec_items = 10, show = True):
    '''
    Function to produce user recommendations
    Required Input - 
        - model = Trained matrix factorization model
        - interactions = dataset used for training the model
        - user_id = user ID for which we need to generate recommendation
        - user_dict = Dictionary type input containing interaction_index as key and user_id as value
        - item_dict = Dictionary type input containing item_id as key and item_name as value
        - threshold = value above which the rating is favorable in new interaction matrix
        - nrec_items = Number of output recommendation needed
    Expected Output - 
        - Prints list of items the given user has already bought
        - Prints list of N recommended items  which user hopefully will be interested in
    '''
    n_users, n_items = interactions.shape
    user_x = user_dict[user_id]
    scores = pd.Series(model.predict(user_x,np.arange(n_items)))
    scores.index = interactions.columns
    scores = list(pd.Series(scores.sort_values(ascending=False).index))
    
    known_items = list(pd.Series(interactions.loc[user_id,:] \
                                 [interactions.loc[user_id,:] > threshold].index) \
								 .sort_values(ascending=False))
    
    scores = [x for x in scores if x not in known_items]
    return_score_list = scores[0:nrec_items]
    known_items = list(pd.Series(known_items).apply(lambda x: item_dict[x]))
    scores = list(pd.Series(return_score_list).apply(lambda x: item_dict[x]))
    if show == True:
        print("Known Products Bought:")
        counter = 1
        for i in known_items:
            print(str(counter) + '- ' + i)
            counter+=1

        print("\n Recommended Products:")
        counter = 1
        for i in scores:
            print(str(counter) + '- ' + i)
            counter+=1
    return return_score_list
    

def sample_recommendation_item(model,interactions,item_id,user_dict,item_dict,number_of_user):
    '''
    Funnction to produce a list of top N interested users for a given item
    Required Input -
        - model = Trained matrix factorization model
        - interactions = dataset used for training the model
        - item_id = item ID for which we need to generate recommended users
        - user_dict =  Dictionary type input containing interaction_index as key and user_id as value
        - item_dict = Dictionary type input containing item_id as key and item_name as value
        - number_of_user = Number of users needed as an output
    Expected Output -
        - user_list = List of recommended users 
    '''
    n_users, n_items = interactions.shape
    x = np.array(interactions.columns)
    scores = pd.Series(model.predict(np.arange(n_users), np.repeat(x.searchsorted(item_id),n_users)))
    user_list = list(interactions.index[scores.sort_values(ascending=False).head(number_of_user).index])
    return user_list 


def create_item_emdedding_distance_matrix(model,interactions):
    '''
    Function to create item-item distance embedding matrix
    Required Input -
        - model = Trained matrix factorization model
        - interactions = dataset used for training the model
    Expected Output -
        - item_emdedding_distance_matrix = Pandas dataframe containing cosine distance matrix b/w items
    '''
    df_item_norm_sparse = sparse.csr_matrix(model.item_embeddings)
    similarities = cosine_similarity(df_item_norm_sparse)
    item_emdedding_distance_matrix = pd.DataFrame(similarities)
    item_emdedding_distance_matrix.columns = interactions.columns
    item_emdedding_distance_matrix.index = interactions.columns
    return item_emdedding_distance_matrix

def item_item_recommendation(item_emdedding_distance_matrix, item_id, 
                             item_dict, n_items = 10, show = True):
    '''
    Function to create item-item recommendation
    Required Input - 
        - item_emdedding_distance_matrix = Pandas dataframe containing cosine distance matrix b/w items
        - item_id  = item ID for which we need to generate recommended items
        - item_dict = Dictionary type input containing item_id as key and item_name as value
        - n_items = Number of items needed as an output
    Expected Output -
        - recommended_items = List of recommended items
    '''
    recommended_items = list(pd.Series(item_emdedding_distance_matrix.loc[item_id,:]. \
                                  sort_values(ascending = False).head(n_items+1). \
                                  index[1:n_items+1]))
    if show == True:
        print("Item of interest: {0}".format(item_dict[item_id]))
        print("Item similar to the above item:")
        counter = 1
        for i in recommended_items:
            print(str(counter) + '- ' +  item_dict[i])
            counter+=1
    return recommended_items

In [21]:
purchases = df[['order_id', 'category', 'GL_Margin']]

In [22]:
purchases.shape

(1172420, 3)

In [23]:
from sklearn.preprocessing import LabelEncoder

In [24]:
purchases.sort_values("category")

,order_id,category,GL_Margin
28339,OR_8D4Z8KY,Artprint,5.60
225554,OR_811C4A1,Artprint,3.80
24578,OR_89PDCDZ,Artprint,7.00
24759,OR_89N13N8,Artprint,3.50
25016,OR_89C3F5Q,Artprint,3.50
25435,OR_89HS3AZ,Artprint,3.50
25831,OR_89KWFZ1,Artprint,2.80
25964,OR_881XCA7,Artprint,3.50
26084,OR_8ADH363,Artprint,2.80
26146,OR_8ATC509,Artprint,3.50


In [25]:
new_cat = LabelEncoder()
purchases["cat_code"] = new_cat.fit_transform(purchases["category"])
purchases[["category", "cat_code"]].head(30)

,category,cat_code
0,Unisex T- Shirts S/S,27
1,Unisex T- Shirts S/S,27
2,Unisex T- Shirts L/S,26
3,Unisex T- Shirts S/S,27
4,Unisex T- Shirts S/S,27
5,Unisex T- Shirts S/S,27
6,Unisex T- Shirts S/S,27
7,Unisex T- Shirts S/S,27
8,Unisex T- Shirts S/S,27
9,Womens T-shirts S/S,30


In [26]:
purchases.head()

,order_id,category,GL_Margin,cat_code
0,ahBzfmdlYXJsYXVuY2gtaHVicg8LEgVPcmRlchjqo9foDgw,Unisex T- Shirts S/S,10.46,27
1,ahBzfmdlYXJsYXVuY2gtaHVicg8LEgVPcmRlchiooI_oDgw,Unisex T- Shirts S/S,13.48,27
2,ahBzfmdlYXJsYXVuY2gtaHVicg8LEgVPcmRlchi00Y7vDgw,Unisex T- Shirts L/S,11.47,26
3,ahBzfmdlYXJsYXVuY2gtaHVicg8LEgVPcmRlchjStcDxDgw,Unisex T- Shirts S/S,13.48,27
4,ahBzfmdlYXJsYXVuY2gtaHVicg8LEgVPcmRlchiQpOz3Dgw,Unisex T- Shirts S/S,10.18,27


In [27]:
purchases2 = purchases[['order_id', 'cat_code', 'GL_Margin']]

In [28]:
purchases2.head()

,order_id,cat_code,GL_Margin
0,ahBzfmdlYXJsYXVuY2gtaHVicg8LEgVPcmRlchjqo9foDgw,27,10.46
1,ahBzfmdlYXJsYXVuY2gtaHVicg8LEgVPcmRlchiooI_oDgw,27,13.48
2,ahBzfmdlYXJsYXVuY2gtaHVicg8LEgVPcmRlchi00Y7vDgw,26,11.47
3,ahBzfmdlYXJsYXVuY2gtaHVicg8LEgVPcmRlchjStcDxDgw,27,13.48
4,ahBzfmdlYXJsYXVuY2gtaHVicg8LEgVPcmRlchiQpOz3Dgw,27,10.18


In [29]:
items = des[['cat_code', 'category', 'super_category']]

In [30]:
items.head(50)

,cat_code,category,super_category
0,1,Artprint,Wall Art
1,2,Baby,Apparel
2,3,Bangle,Jewelry
3,4,Bedroom,Home
4,5,Canvas,Wall Art
5,6,Dog Tag,Jewelry
6,7,Frameart,Wall Art
7,8,Galaxy Flexi Case,Tech Accessories
8,9,Galaxy Slim Case,Tech Accessories
9,10,Galaxy Tough Case,Tech Accessories


In [31]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 3 columns):
cat_code          30 non-null int64
category          30 non-null object
super_category    30 non-null object
dtypes: int64(1), object(2)
memory usage: 792.0+ bytes


In [32]:
interactions = create_interaction_matrix(df = purchases2,
                                         user_col = 'order_id',
                                         item_col = 'cat_code',
                                         rating_col = 'GL_Margin',
                                         threshold = '3')
interactions.shape



(889783, 34)

In [33]:
interactions.head()

cat_code,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,33
order_id,,,,,,,,,,,,,,,,,,,,,
OR_48PWYGT,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,5.21,0.00,0.00,0.00,0.00,0.00,0.00,0.00
OR_4RNBHP1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,3.81,0.00,0.00,0.00
OR_4S84P33,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,4.14,0.00,0.00,0.00,0.00,0.00,0.00
OR_4V7TGD5,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,6.78,0.00,0.00,0.00,4.82,0.00,0.00,0.00
OR_4VCNZVG,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,2.84,0.00,0.00,0.00,0.00,0.00,0.00


In [34]:
user_dict = create_user_dict(interactions=interactions)


In [35]:
items_dict = create_item_dict(df = items,
                               id_col = 'cat_code',
                               name_col = 'category')

In [36]:
mf_model = runMF(interactions = interactions,
                 n_components = 30,
                 loss = 'warp',
                 k = '15',
                 epoch = 30,
                 n_jobs = 4)



## User Recommender

In [39]:
rec_list = sample_recommendation_user(model = mf_model, 
                                      interactions = interactions, 
                                      user_id = 'OR_855RQYK', 
                                      user_dict = user_dict,
                                      item_dict = items_dict, 
                                      threshold = 10,
                                      nrec_items = 2)

Known Products Bought:
1- Canvas

 Recommended Products:
1- Womens T-shirts L/S
2- Womens T-shirts S/S


## User - Item Recommender

In [38]:
sample_recommendation_item(model = mf_model,
                           interactions = interactions,
                           item_id = 1,
                           user_dict = user_dict,
                           item_dict = items_dict,
                           number_of_user = 15)

['OR_8E81ZBA',
 'OR_86FJ3T6',
 'OR_86HTNXY',
 'OR_816A4N4',
 'OR_84DQ4Y8',
 'OR_80NG51G',
 'OR_80XG35G',
 'OR_7Z2Z7XH',
 'OR_8DSTP4S',
 'OR_7Y95SG3',
 'OR_7YP0NYV',
 'OR_814RMFY',
 'OR_80ACCZ7',
 'OR_883CAWM',
 'OR_B106WDM']

## Item Item Recommender

In [40]:
item_item_dist = create_item_emdedding_distance_matrix(model = mf_model,
                                                       interactions = interactions)

In [43]:
rec_list = item_item_recommendation(item_emdedding_distance_matrix = item_item_dist,
                                    item_id = 1,
                                    item_dict = items_dict,
                                    n_items = 2)



Item of interest: Artprint
Item similar to the above item:
1- Poster
2- Travel Mug
